In [8]:
# Cellule 1: Imports et Configuration

import pandas as pd
import numpy as np
import os
import joblib
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import warnings

# Ignorer les avertissements futurs pour garder une sortie propre
warnings.filterwarnings('ignore', category=FutureWarning)

print("Librairies importées avec succès.")

Librairies importées avec succès.


In [11]:
# Cellule 2: Génération du Dataset Synthétique (Version Corrigée)

print("Génération du jeu de données d'entraînement synthétique...")

# ... (toute la partie génération de données reste la même) ...
n_exemples = 1000
data = []
for _ in range(n_exemples):
    hs_minutes = np.random.randint(10, 240)
    taux_absence = np.random.rand() * 0.6
    historique_hs = np.random.randint(0, 25)
    score = 0
    if taux_absence > 0.4: score += 2
    if historique_hs > 10: score += 2
    if hs_minutes > 180: score -= 2
    if hs_minutes < 30: score += 1
    decision_reelle = 1 if score >= 2 else 0
    data.append([hs_minutes, taux_absence, historique_hs, decision_reelle])
columns = ['hs_minutes', 'taux_absence_service', 'nb_hs_validees_historique', 'decision_reelle']
df = pd.DataFrame(data, columns=columns)

# --- CORRECTION DU CHEMIN DE SAUVEGARDE ---
# S'assurer que le dossier 'models' existe
if not os.path.exists('./models'):
    os.makedirs('./models')

# Sauvegarder le dataset DANS le dossier 'models'
dataset_path = './models/dataset_hs_contextuel.csv' # <-- CHEMIN CORRIGÉ
df.to_csv(dataset_path, index=False)

print(f"-> Dataset de {len(df)} lignes créé et sauvegardé dans '{dataset_path}'")
print("\nAperçu des 5 premières lignes du dataset :")
print(df.head())

Génération du jeu de données d'entraînement synthétique...
-> Dataset de 1000 lignes créé et sauvegardé dans './models/dataset_hs_contextuel.csv'

Aperçu des 5 premières lignes du dataset :
   hs_minutes  taux_absence_service  nb_hs_validees_historique  \
0          89              0.361278                         24   
1         100              0.447711                          9   
2         136              0.117052                          0   
3          97              0.123559                          3   
4         230              0.229517                         10   

   decision_reelle  
0                1  
1                1  
2                0  
3                0  
4                0  


In [12]:
# Cellule 3: Entraînement et Sauvegarde du Modèle (Version Corrigée)

print("\nDébut du processus d'entraînement...")

# --- CORRECTION DU CHEMIN DE LECTURE ---
# Définir le chemin vers le dataset que nous venons de créer
dataset_path = './models/dataset_hs_contextuel.csv' # <-- CHEMIN CORRIGÉ

# 1. Charger le dataset
df_train = pd.read_csv(dataset_path)

# 2. Séparer les 'features' et la 'target'
X = df_train.drop('decision_reelle', axis=1)
y = df_train['decision_reelle']

# 3. Diviser les données pour l'entraînement et pour le test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"Données divisées : {len(X_train)} pour l'entraînement, {len(X_test)} pour le test.")

# 4. Créer et entraîner le modèle
print("\nEntraînement du modèle...")
model = lgb.LGBMClassifier(random_state=42)
model.fit(X_train, y_train)
print("-> Modèle entraîné.")

# 5. Évaluer la performance
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nPrécision du modèle sur le set de test : {accuracy:.2%}")
print("\nRapport de classification détaillé :")
print(classification_report(y_test, y_pred, target_names=['REJETER (0)', 'ACCEPTER (1)']))

# 6. Sauvegarder le modèle entraîné (le "cerveau") dans le dossier 'models'
model_path = './models/model_hs_contextuel.pkl'
joblib.dump(model, model_path)

print(f"\n--- SUCCÈS ---")
print(f"Le modèle entraîné a été sauvegardé ici : '{model_path}'")


Début du processus d'entraînement...
Données divisées : 800 pour l'entraînement, 200 pour le test.

Entraînement du modèle...
[LightGBM] [Info] Number of positive: 439, number of negative: 361
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000086 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 444
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.548750 -> initscore=0.195621
[LightGBM] [Info] Start training from score 0.195621
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa